# Swarmalators with Stochastic Coupling and Memory
Simulation<br>
D. Kofler, 2022

This simulation was created based on the paper 'Swarmalators with Stochastic Coupling and Memory' by Udo Schilcher, Jorge F. Schmidt, Arke Vogell, and Christian Bettstetter<br>
http://data.bettstetter.com/publications/schilcher-2021-acsos-swarmalators.pdf

### Imports

In [ ]:
from swarmalator_model.simulation import Simulation
from swarmalator_model.simulation_run import Simulation_run
from swarmalator_model.preset import Preset
from swarmalator_model.analysis import Analysis
from swarmalator_model import helper_functions as hlp

### Run Simulation

Different patterns emerge when swarmalators converge depending on *J* and *K*.

|Pattern               |   J  |   K   |
|----------------------|------|-------|
|Static sync           |  0.1 |   1   |
|static async          |  0.1 |  -1   |
|static phase wave     |    1 |   0   |
|splintered phase wave |    1 | -0.1  |
|active phase wave     |    1 | -0.75 |

In [ ]:
sim = Simulation(
    plot_size=1500,
    logging=True,
    num_swarmalators=100,
    memory_init='random',
    coupling_probability=0.1,
    J=1,
    K=0,
    alpha=0.5)
sim.run_simulation()

In [ ]:
preset1 = Preset()
preset1.load_from_json('100_random_0.1_1.0_0.0_0.1_0.0.json')
preset1.summary()

In [ ]:
preset2 = Preset()
preset2.load_from_json('100_random_0.1_1.0_0.0_0.1_0.5.json')
preset2.summary()

In [ ]:
presets = [preset1, preset2]
Simulation_run(presets=presets, sim_time=5).start()

### Analyze Simulation Data

In [ ]:
file1 = 'dataset_1.0_0.0_0.1_0.0_20220924203958.ssd'
dataset1 = hlp.load_data('sim_data\\' + file1)
dataset1.summary()

In [ ]:
file2 = 'dataset_1.0_0.0_0.1_0.5_20220924204004.ssd'
dataset2 = hlp.load_data('sim_data\\' + file2)
dataset2.summary()

In [ ]:
a = Analysis()
a.add_dataset(dataset1, 'alpha_0')
a.add_dataset(dataset2, 'alpha_0_5')
a.list_datasets()

In [ ]:
a.plot_avg_speed('alpha_0')

In [ ]:
a.plot_avg_speed('alpha_0_5')